# MISP & PyMISP - Threat Intelligence Analysis Workshop

## What is MISP?

MISP is an open-source threat intelligence platform designed for sharing, storing, and correlating Indicators of Compromise (IoCs) and threat intelligence data. Originally developed by CIRCL, MISP has become a cornerstone of collaborative cybersecurity efforts worldwide.

## Core MISP Concepts and Terminology

### Events and Attributes
- **Events**: Container objects that represent security incidents, campaigns, or threat intelligence reports
- **Attributes**: Individual pieces of threat intelligence (IP addresses, domains, file hashes, URLs, etc.)
- **Objects**: Complex structures that group related attributes together (e.g., a file object containing hash, filename, and size)
- **Relationships**: Connections between events, attributes, and objects that show how threats relate to each other

### MISP Galaxies and Taxonomies
- **Taxonomies**: Classification systems used to tag and categorize threat intelligence
- **Tags**: Labels applied to events and attributes using taxonomies and galaxy information
- **Galaxies**: Knowledge bases containing structured threat intelligence (threat actors, attack patterns, tools, etc.)
- **Clusters**: Specific entries within galaxies (e.g., "APT29" within the threat-actor galaxy)

### MITRE ATT&CK Integration
- **Techniques**: Specific attack methods catalogued in the MITRE ATT&CK framework
- **Tactics**: High-level categories of attack behavior (e.g., Persistence, Defense Evasion)
- **Procedures**: Specific implementations of techniques by threat actors
- **Sub-techniques**: More granular variations of main techniques

## PyMISP Library

PyMISP is the official Python library for interacting with MISP instances:
- **API Wrapper**: Provides Python methods for all MISP API endpoints
- **Object-Oriented**: Represents MISP data as Python objects for easy manipulation
- **Search Capabilities**: Powerful querying with filters, boolean logic, and temporal ranges
- **Data Export**: Multiple formats for integration with other security tools

## Key Features of MISP

### Threat Intelligence Sharing
- **Community Collaboration**: Share IOCs and threat data across organizations
- **Automated Feeds**: Consume and distribute threat intelligence through standardized formats
- **Access Control**: Granular permissions for data sharing and visibility

### Data Correlation
- **Automatic Correlation**: MISP automatically identifies relationships between similar attributes
- **Manual Relationships**: Analysts can create explicit connections between threats
- **Contextual Enrichment**: Add metadata and context to raw threat indicators

### Integration Capabilities
- **API Access**: RESTful API for programmatic access and automation
- **Export Formats**: STIX, CSV, JSON, XML, and custom formats
- **SIEM Integration**: Direct feeds to security information and event management systems

## Documentation

- API Specs: https://www.misp-project.org/openapi/
- Cheatsheet: https://www.misp-project.org/misp-training/cheatsheet.pdf
- MISP REST API: https://github.com/MISP/misp-training/blob/main/a.7-rest-API/Training%20-%20Using%20the%20API%20in%20MISP.ipynb
    - query: https://github.com/MISP/misp-training/blob/main/a.7-rest-API/query-misp.ipynb
- PyMISP: https://github.com/MISP/PyMISP/blob/main/docs/tutorial/FullOverview.ipynb

# Using the PyMISP objects

PyMISP is the python library used to deal with MISP format so you do not have to deal with the JSON blob yourself without knowing about the required and optional fields.

## MISPEvent

MISPEvent is the main class to use when you want to create/update events on a MISP instance.

In [ ]:
from pymisp import MISPEvent

event = MISPEvent()

event.info = 'A fancy MISP Event'  # Required
event.distribution = 0  # Optional, sets the distribution level to "Your Organisation only"
event.threat_level_id = 2  # Optional, sets the threat level to "Medium"
event.analysis = 1  # Optional, set the analysis to "Ongoing"

print(event.to_json(indent=4))

### Set the event date

The date can be in many different formats. PyMISP normalizes it in a way that will be understood by your MISP instance: a date in the `YYYY-MM-DD` format.

In [ ]:
event.set_date('2022-01-06')
print(f'From a text date: {event.date}')

from datetime import date
d = date.today()
event.set_date(d)
print(f'From a datetime.date date: {event.date}')

from datetime import datetime
d = datetime.now()
event.set_date(d)
# MISP expects a day, so the Hour will be dropped
print(f'From a datetime.datetime date: {event.date}')

### Tag an Event

An easy way to tag an Event

In [ ]:
event.add_tag('tlp:white')

print(event.to_json(indent=4))

## MISP Attribute

MISP Attributes are the raw pieces of data that can either be indicators of compromise (IoCs) or observed data.

They are defined with a triplet of required values being a `type`, a `category` and a `value`, and a bunch of optional fields.

The Attributes parameters are the following:
- **type** (required)
- **value** (required)
- **category**: the type of information (if not set, the default one for the given `type` is used)
- **to_ids**: defines whether the attribute defines some malicious data that should be blocked, or not (if not set, the default value for the given `type` is used)
- **distribution**: defaults to inherit from parent (event or object)
- **disable_correlation**: if you want to avoid correlations between events on that specific value
- **data**: for `malware-sample` and `attachment` types, BytesIO object of the file.

### A minimal and quick way of adding an attribute

In [ ]:
attribute1 = event.add_attribute('ip-dst', '8.8.8.8')

print(attribute1.to_json(indent=4))

### Set inline parameters

In [ ]:
attribute2 = event.add_attribute('ip-dst', '8.8.8.9', to_ids=False, disable_correlation=True)

print(attribute2.to_json(indent=4))

### Modify an existing attribute

Every parameter can be modified in a pythonic way

In [ ]:
attribute1.to_ids = False

print(attribute1.to_json(indent=4))

### Soft delete an attribute

The default approach on MISP is to soft delete data, which means it will not be displayed in the dafult view on MISP.  
The reason for doing this is to allow to push delete updates to instances we synchronise with.

In [ ]:
attribute2.delete()

print(event.to_json(indent=4))

### A more advanced way of passing the different parameters at once

In [ ]:
from uuid import uuid4

attribute_uuid = uuid4()
print(attribute_uuid)

kwargs = {
    'to_ids': False,
    'disable_correlation': True,
    'category': 'Network activity',
    'uuid': attribute_uuid
}
attribute = event.add_attribute('ip-src', '1.1.1.1', **kwargs)

print(attribute.to_json(indent=4))

### Using the MISPAttribute class

Allows you to play with the attribute before adding it to the event.

It is then possible to load the attribute from a JSON or from a dict

In [ ]:
from pymisp import MISPAttribute

attribute = MISPAttribute()
attribute.type = 'domain'
attribute.value = 'circl.lu'

print(attribute.to_json(indent=4))
print(event.add_attribute(**attribute).to_json(indent=4))

In [ ]:
# Loaded from a JSON
json = '''{
    "type": "domain",
    "value": "circl.lu",
    "to_ids": false
}'''

json_attribute = MISPAttribute()
json_attribute.from_json(json)

print(json_attribute.to_json(indent=4))

In [ ]:
# Loaded from a python dict
_dict = {
    'type': 'domain',
    'value': 'circl.lu',
    'to_ids': False
}

dict_attribute = MISPAttribute()
dict_attribute.from_dict(**_dict)

print(dict_attribute.to_json(indent=4))

### Tag an Attribute

The same way to tag events applies for attributes

In [ ]:
dict_attribute.add_tag('tlp:clear')

print(dict_attribute.to_json(indent=4))

## MISP Object

MISP Objects are containers to group attributes in a way that makes sense. The objects are based on templates that are bundled in the library itself, but you can also use your own templates.



In [ ]:
from pymisp import MISPObject

misp_object = MISPObject('domain-ip')
misp_object.comment = 'My fancy new object'

object_attribute = misp_object.add_attribute('domain', value='circl.lu')
object_attribute.add_tag('tlp:green')
misp_object.add_attribute('ip', value='149.13.33.14')
misp_object.add_attribute('first-seen', value='2022-12-31')
misp_object.add_attribute('last-seen', value='2023-01-06')

print(misp_object.to_json(indent=4))

### Short version to add an object to a MISPEvent

You can also add the object directly in a MISP event

In [ ]:
from pymisp import MISPObject

misp_object = event.add_object(name='domain-ip', comment='My fancy new object')

object_attribute = misp_object.add_attribute('domain', value='circl.lu')
object_attribute.add_tag('tlp:green')
misp_object.add_attribute('ip', value='149.13.33.14', to_ids=False)
misp_object.add_attribute('first-seen', value='2022-12-31', disable_correlation=False)
misp_object.add_attribute('last-seen', value='2023-01-06')

misp_object.add_reference(attribute1.uuid, 'connects-to')

print(event.to_json(indent=4))

# Helpers for MISPObjects

For some objects, we have helpers in order to make your life easier. The most relevant example is the file object: when you have a file to push on MISP, there are plenty of indicators you can extract at once, and it is pretty simple to automate, so we made it a oneliner.

**Note**: This requires a few more dependencies to get the full power of the script: 
* `lief` to extract indicators out of PE/ELF/MachO files, and soon Android binaries.
* `python-magic` to get the mime type
* `pydeep` to compute the ssdeep of the binary whenever possible


```bash
pip install lief python-magic git+https://github.com/kbandla/pydeep.git
```

In [ ]:
from pathlib import Path
test_path = Path().resolve().parent / 'files'
print(test_path)

from pymisp.tools import FileObject

file_object = FileObject(
    filepath=test_path / 'EICAR.com',
    standalone=False
)
print(file_object.to_json(indent=4))

## Playing with a malware sample

The data you receive out of the JSON dump from a MISP instance is a base64 encoded zip with `infected` as a password.  
The zip file contains 2 files, one containing the original file name of the uploaded file, and the other one is the binary.

This is pretty much a pain to use as-is, so there is an helper for that!

In [ ]:
sample = file_object.get_attributes_by_relation('malware-sample')[0]
print(sample)
print('File name --->', sample.malware_filename)
print(sample.malware_binary)
print('Content of the malware (in bytes) ----->', sample.malware_binary.getvalue())

## Use lief to extract indicators out of binaries

Another cool hepler allows you to pass the path of a binary. If the binary's format is supported by `lief`, you get the file object, the binary definition (PE, ELf or Mach-o) and the relevant sections.

If it is anything else, it will simply generate a file object.

In [ ]:
from pymisp.tools import make_binary_objects

misp_event = MISPEvent()
misp_event.info = 'Test with binary file'

filepath = test_path / 'whoami.exe'
file_obj, bin_obj, sections = make_binary_objects(
    filepath=filepath.as_posix(),
    standalone=False
)

file_object = misp_event.add_object(file_obj)
if bin_obj:
    misp_event.add_object(bin_obj)
    for section in sections:
        misp_event.add_object(section)

The references between the different objects are also set by default with the `make_binary_objects` method.

In [ ]:
for misp_object in misp_event.objects:
    print(misp_object)

### Add references between Objects

MISP supports references between Objects, as well as references between an Object and an Attribute.

References between MISP Attributes are on the other hand not currently supported.

In [ ]:
# Encoding an e-mail
email = MISPObject('email')
email.add_attribute('from', 'andrew_ryan@rindustries.rp')
email.add_attribute('subject', 'Report for case 4829-2375')
email.add_attribute('email-body', 'Please see the attached Iolta report for 4829-2375.\r\n\r\nWe received a check request in the amount of $19,637.28 for the above referenced file. However, the attached report refects a $0 balance. At your earliest convenience, please advise how this request is to be funded.\r\n\r\nThanks.\r\n\r\nAndrew_Ryan *\r\nAccounts Payable\r\n\r\nRyan Industries\r\n42, Central Control Hephaestus - Rapture\r\nwww.rindustries.rp\r\n\r\n*Not licensed to practise law.\r\n\r\nThis communication contains information that is intended only for the recipient named and may be privileged, confidential, subject to the attorney-client privilege, and/or exempt from disclosure under applicable law. If you are not the intended recipient or agent responsible for delivering this communication to the intended recipient, you are hereby notified that you have received this communication in error, and that any review, disclosure, dissemination, distribution, use, or copying of this communication is STRICTLY PROHIBITED. If you have received this communication in error, please notify us immediately by telephone at 1-800-766-7751 or 1-972-643-6600 and destroy the material in its entirety, whether in electronic or hard copy format.')
email_object = event.add_object(email)

# Encoding the Identity of the sender of the e-mail
person = MISPObject('person')
person.comment = 'The sender of the email'
person.add_attribute('full-name', 'Andrew Ryan')
person.add_attribute('e-mail', 'andrew_ryan@rindustries.rp')
person.add_attribute('role', 'Suspect')
person_object = event.add_object(person)

# Encoding an IP address used to connect to a C2 server distributing ransomware
ip_attribute = MISPAttribute()
ip_attribute.from_dict(
    **{
        'type': 'ip-dst',
        'value': '81.177.170.166',
        'comment': 'IP address of a C2 server distributing the ransomware'
    }
)
added_ip = event.add_attribute(**ip_attribute)

#### Adding references between the Objects we just created

In [ ]:
# From the email object, add a reference to the person object
email_object.add_reference(person_object.uuid, 'sent-by')

# From the file object, add a reference to the email object
file_object.add_reference(email_object.uuid, 'dropped-by')

# From the file_object, add a reference to the IP address Attribute
file_object.add_reference(ip_attribute.uuid, 'connects-to')

In [ ]:
print(email_object)
for reference in email_object.references:
    print(f' - {reference.relationship_type} {reference.referenced_uuid}')
print(file_object)
for reference in file_object.references:
    print(f' - {reference.relationship_type} {reference.referenced_uuid}')

## CSV support

In [ ]:
valid_csv = test_path / 'valid_fieldnames.csv'

with open(valid_csv, 'r') as f:
    print(f.read())

In this case, we have valid field names, we can use the file as is and pass it to the CSV loader:

In [ ]:
from pymisp.tools import CSVLoader

csv1 = CSVLoader(template_name='file', csv_path=valid_csv)
csv_event = MISPEvent()
csv_event.info = 'Test event from CSV loader'

for o in csv1.load():
    csv_event.add_object(**o)
    
print(csv_event.to_json(indent=4))

In [ ]:
invalid_csv = test_path / 'invalid_fieldnames.csv'

with open(invalid_csv, 'r') as f:
    print(f.read())

The field names being invalid, we have to specify them with some valid `object_relation` fields.

In our case we also have to tell the CSV loader that we already have field names otherwise the first line is going to be imported in a MISP object.

In [ ]:
csv_event = MISPEvent()
csv_event.info = 'Test event from CSV loader'

csv2 = CSVLoader(
    template_name='file',
    csv_path=invalid_csv,
    fieldnames=['sha1', 'filename', 'size-in-bytes'],
    has_fieldnames=True
)

for o in csv2.load():
    csv_event.add_object(**o)
    
print(csv_event.to_json(indent=4))

## Generic helper

This helper can be used when you already have a script that does the mapping between your own code and a MISPObject template.

In [ ]:
from pymisp.tools import GenericObjectGenerator

attributes_as_dict = [
    {
        'filename': 'shell1.exe',
        'sha1': {
            'value': 'b7afa7acf1b7ded2c4e3d0884b5cdaa230d9f82e',
            'to_ids': False
        },
        'size-in-bytes': {
            'value': 24576,
            'disable_correlation': True
        }
    }
]

misp_object = GenericObjectGenerator('file', strict=True)
misp_object.generate_attributes(attributes_as_dict)

print(misp_object.to_json(indent=4))

### User defined objects

The Generic helper can also be used to define your own object template.

In [ ]:
attributes_as_dict = [
    {
        'MyCoolAttribute': {
            'value': 'critical thing',
            'type': 'text'
        },
        'MyCoolerAttribute': {
            'value': 'even worse',
            'type': 'text'
        }
    }
]

# We cannot use `strict=True` here
misp_object = GenericObjectGenerator('my-cool-template')
misp_object.generate_attributes(attributes_as_dict)

print(misp_object.to_json(indent=4))

PyMISP is OK with this generic object and won't complain if you set the required fields.

Nonetheless, before pushing such event to MISP, we want to set a few additional fields:

In [ ]:
from uuid import uuid4

misp_object.template_uuid = uuid4()
misp_object.template_id = 1
misp_object.description = 'foo'
setattr(misp_object, 'meta-category', 'bar')

print(misp_object.to_json(indent=4))

### Use locally defined object templates

**Important**: The path you pass as parameter for `misp_objects_path_custom` needs to contain a directory equals to the value of the parameter `name` (same structure as the content of the `misp-object` repository)

In [ ]:
user_defined_obj = MISPObject(
    name='test_object_template',
    strict=True,
    misp_objects_path_custom=test_path
)

user_defined_obj.add_attribute('member1', 'foo')
user_defined_obj.add_attribute('member2', value='bar', to_ids=True)
user_defined_obj.add_attribute('member3', **{'value': 'baz'})

print(user_defined_obj.to_json(indent=4))